In [ ]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()



In [ ]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

(cont:sync:criticalsection)=
# The Critical Section Problem

The problem of controlling concurrent access to shared data was recognized very early. In an [undated lecture]`https://www.cs.utexas.edu/users/EWD/translations/EWD35-English.html` from 1962 or 1963, Dutch computer scientist Edsger W. Dijkstra described what has come to known as the *critical section problem*.  Each thread executes some *critical section* of code that must be guarded against concurrent execution, i.e., only one thread is allowed to be executing in its critical section at any given time. The problem is to design a protocol that threads must execute prior to entering their critical section that will enforce the single-access rule. The term *critical section* is unfortunately misleading, as it should be obvious that it is the *shared data* that must be protected from concurrent access, and not the code. Think of the critical section as the code that must execute without interference in order to use the shared data correctly. In the shared counter example, the loop body in the `increment_thread()` and `decrement_thread()` functions are the critical sections for the counter variable. To identify the critical sections of code, we need to identify all the cases where shared data must be protected against simultaneous modification or access to prevent race conditions. Note also that the same code can be invoked to operate on different data -- for example, `deposit()` called for account \#100  in Thread 1, and `deposit()` called for account \#200 in Thread 2 will not interfere with each other. In this case, we have critical sections for two different resources (i.e., two different bank accounts) and they do not need to be protected from each other. 

These ideas will become more concrete by the end of this Chapter, but initially we will reason about a single shared resource (like our `counter` variable), accessed repeatedly by multiple threads, with some additional code before and after the critical section that does not make any accesses to the shared variable. 


## Problem Definition

In its classical form, a solution to the critical section problem must meet the following requirements:

Mutual Exclusion
 : Only one thread can be executing in its critical section at a time.

Progress
 : If no thread is executing in its critical section, and some threads want to enter their critical section, then eventually some thread must be granted entry to its critical section. Threads that are executing in their *remainder* code (i.e., code that is unrelated to the critical section) must not delay threads that want to enter the critical section. 
 
Bounded Waiting (No Starvation)
 : Once a thread has requested entry to its critical section, there is a limit on how many times other threads are allowed to enter their critical sections ahead of it. 
 
The *mutual exclusion* requirement is fairly obvious---we want to ensure that only one thread at a time can be manipulating the shared data. The *progress* requirement disallows solutions that meet the mutual exclusion requirement by not allowing *any* thread to enter the critical section. Finally, the *bounded waiting* requirement ensures that every thread has a fair chance to use the shared data.

To these basic requirements, we will add a fourth consideration:

Performance
 : The overhead of entering and exiting the critical section should be small with respect to the work being done within it.

In some solutions that we will look at, the bounded waiting requirement is not met, but it is statistically unlikely in real systems that a thread will be continually passed over in favor of other threads entering the critical section. 

We also make a number assumptions:

1. Each thread is executing at non-zero speed, but we make no assumptions about the relative speed of the threads.
2. We assume that individual machine instructions such as `load`, `store`, or `add` are *atomic*, meaning that they are indivisible. The execution of multiple threads is interleaved at the level of individual machine instructions. For example, if Thread 1 reads a variable and Thread 2 writes the variable at nearly the same time, then Thread 1 will read either the old value, or the new value, but not some mixture of the old and new values. 
3. The operations in each thread are executed in the order that they appear in the program code, and the result of concurrent execution is equivalent to some interleaving of operations from each of the concurrent threads. (Note that this last assumption is not true of modern cpu architectures, but we will omit the topic of memory consistency models for now.)

With these requirements and assumptions in place, we now turn our attention to designing a protocol that threads can use to coordinate their use of shared data. Once we have identified the critical sections, we insert an *entry section* before the critical section, and an *exit section* immediately after the critical section. Each thread must request permission to enter its critical section, by executing the protocol in its entry section. When it completes the entry, the thread is guaranteed to have exclusive access to the shared data accessed within the critical section. When it finishes the critical section, the thread executes the code in the exit section to make the critical section available to other threads. Each thread may request entry to its critical section repeatedly. All other code executed by the thread, either before entry to, or after exit from, the critical section will be referred to as the *remainder* of the code.

{numref}`fig:sync:criticalsection` illustrates these concepts with a concrete code example from the `increment_thread` function in the shared counter program. In this figure, we are showing where the entry and exit sections need to be added around the critical section of code that accesses the shared variable, without specifying what goes into those sections.

```{figure} ../images/sync/critical_section.drawio.png
---
width: 85%
name: fig:sync:criticalsection
---
Abstract view of a critical section of code, surrounded by entry and exit sections (left), and example of these concepts in the `increment_thread` function.
```

Now let's try to implement the protocol for the entry and exit sections. In the following examples, we will use C/C++-like pseudo-code, rather than actual C, to make the key points more clearly. We will also consider the special case of two concurrent threads first, before the general case of N concurrent threads.

We will encapsulate our solutions into a *lock* abstract data type, with the following operations:

- `acquire()` : Obtain exclusive access to a critical region.
- `release()` : Leave the critical region, making it available to other threads.

The lock object may also have some private data, which can be used by the `acquire()` and `release()` functions, but is not otherwise visible. Exactly what this data might be will depend on the implementation of these functions.

We associate a lock object with the shared data that we need to protect, and invoke the `lock.acquire()` function to enter the critical section, and the `lock.release()` function to leave the critical section, as shown in {numref}`fig:sync:criticalsection_withlock`. Note that we can have multiple lock objects that protect different data objects, such as a separate lock for each bank account. The entry code simply has to invoke the acquire() and release() operations on the right lock object. 


```{figure} ../images/sync/critical_section_with_lock.drawio.png
---
width: 85%
name: fig:sync:criticalsection_withlock
---We w
Encapsulating critical section entry and exit with operations on a lock object.
```

(cont:sync:criticalsection:interrupt_soln)=
## Critical Section (Broken) Solution 1 - Disable Interrupts

Earlier, we blamed interrupts for causing untimely context switches that allowed thread operations to be interleaved arbitrarily. So, if we disable interrupts in its entry section before the critical section, the currently executing thread cannot be kicked off the CPU until it enables interrupts in the exit section. Alas, this enticingly simple solution won't work in most cases. First, user-level processes are not allowed to disable interrupts, so this solution is only available to operating system code. Second, leaving interrupts disabled for a long period of time can cause other problems, so even in the OS, it can only be used for short critical sections that do not cause blocking. Finally, and perhaps most importantly, disabling interrupts offers no protection against concurrent accesses from a thread running on another CPU, so this solution can only be applied on a uniprocessor. Historically, disabling interrupts was used to protect short critical sections in OS code, but in the multicore era, this idea has very limited uses.

```{code-block} c
:name: listing:sync:cs_interrupts
:caption: Example of critical section entry and exit code using interrupt disabling; C++-like definition.
struct lock {

    void acquire() {
        disable_interrupts(); // cli on x86
    }

    void release() {
        enable_interrupts(); // sti on x86
    }
}
```

(cont:sync:criticalsection:flag_soln)=
## Critical Section (Broken) Solution 2 - Check a Flag

Suppose we introduce a shared boolean variable to check if the critical section is currently available, as shown in {numref}`listing:sync:cs_lockflag`. 

```{code-block} c
:name: listing:sync:cs_lockflag
:caption: Critical section entry and exit code using a flag; C++-like definition.
struct lock {
    bool available = true;
    
    void acquire() {
        while(!available) { }; // empty loop body, just spins
        available = false;
    }

    void release() {
        available = true; 
    }
}
```

Each thread will check the `available` flag repeatedly in the `while()` loop until it finds the critical section is free. When a thread breaks out of the loop, it sets the `available` flag to false, preventing other threads from accessing the critical section until it is done. On exit, it sets the `available` flag to true again so that another thread can enter. 

```{sidebar} This is called a *spinlock* because threads spin around the `while` loop until they can gain entry to the critical section. Since a thread keeps the CPU busy while it is waiting for entry, this spinning is called *busy waiting*.
```

Alas, this does not meet the mutual exclusion requirement. We have just replaced one race with another. Consider two concurrent threads, T0 and T1, that each test the value of `available`, see that is is `true`, and break out of their respective `while` loops. Both T0 and T1 then set `available=false` and proceed to enter their critical sections concurrently.  The mutual exclusion requirement is violated, so we will have to try again. 

The heart of the problem with this non-solution is that our threads' operations can interleave so that the flag can change between the time when a thread tests if it is safe to proceed and the time when the thread sets the flag to prevent others from also entering the critical section. If we could make the TEST and the SET an indivisible atomic operation, this problem would go away. Indeed, today's processor architectures give us an instruction with the properties we need. But, the processors of the Djikstra's era did not, and it is instructive to consider how we can implement the critical section entry and exit protocols without special assistance from the hardware. We will then see how some hardware support simplifies matters considerably.

(cont:sync:criticalsection:turn_soln)=
## Critical Section (Broken) Solution 3 - Take Turns

Let's try replacing the boolean `flag` with a `turn` variable to indicate which thread is allowed to use the critical section. Each thread will wait for `turn` to be set to its own thread id (tid) before entering, and will set `turn` to the other thread's id when it is done, as shown in {numref}`listing:sync:cs_turn`.

```{code-block} c
:name: listing:sync:cs_turn
:caption: Critical section entry and exit code for two threads using turns; C++-like definition.
struct lock {
    int turn = 0;
    
    void acquire() {
        int me = getTid(); // which thread is running this function? Threads are numbered 0 or 1.
        while(turn != me) { }; // empty loop body, just spins
    }

    void release() {
        int me = getTid();
        turn = 1 - me; // set turn to other thread (if me == 0, turn = 1; if me == 1, turn = 0)
    }
}
```

Does this meet our requirements? Let's examine them one by one. 

Mutual exclusion
 : The value of `turn` can only be 0 or 1 at any instant in time, and only the thread whose id matches `turn` is allowed entry to the critical section. So mutual exclusion is satisfied.
 
 Progress
  : This requirement has two parts. The first part says that we can't delay the decision of which thread next gets to enter the critical section forever. That part is fine --- we decide on the next thread in the exit section by flipping the `turn` variable. The second part of the progress requirement says that threads in their *remainder* code can't delay threads that want to use the critical section, and here lies the problem. This solution requires a strict alternation of threads in the critical section. In {numref}`listing:sync:turn_soln}` we initialized `turn` so that T0 must go first, but T0 and T1 could have different tasks and run at different relative speeds. If T1 reaches its critical section first, it is still blocked until T0 goes through its critical section and flips `turn` to 1. Similarly, if one thread has less work to do, and needs fewer trips through the critical section, it could leave the other thread waiting indefinitely for another turn. 
  
Bounded waiting
 : Strangely, this does satisfy bounded waiting, since there is a bound on the number of times other threads can use the critical section after a thread has started the entry protocol. Threads take turns, so T0 can use the critical section at most once after T1 calls the `acquire()` function before T1 gains entry. It is irrelevant though because we don't have Progress. 
 
The issue with this "solution" is that a thread must wait its turn, even when the other thread has no interest in entering its critical section. 

We could go on for quite some time showing you attempts at solutions that do not quite work. In his classic 1965 paper on "Cooperating Sequential Processes" from 1965 {cite}`10.5555/1102034`, Djikstra lays out several other broken alternatives and admits that "people that had played with the problem started to doubt whether it could be solved at all." He gives credit to Dutch mathematication T. J. Dekker for coming up with the first correct solution for two threads (in 1959!). The main idea is to combine the idea of turns with an expression of interest from a thread when it wants to enter its critical section. You can read all about [Dekker's Algorithm]`https://en.wikipedia.org/wiki/Dekker%27s_algorithm` elsewhere, but suffice to say that it was fairly complex and doesn't generalize to more than two threads. 
A simpler solution that can generalize to more than 2 threads, built on the same ideas of combining turns with expressions of interest, was published by Gary L. Peterson in 1981 {cite}`Peterson1981`. 

(cont:sync:criticalsection:peterson_soln)=
## Critical Section Solution 4 - Peterson's Algorithm

The code for Peterson's Algorithm is shown in {numref}`listing:sync:cs_peterson`. We have added an array of boolean flags, `interested` where each thread can indicate its interest in entering its critical section, and initially both flags are set to `false`. Each thread will only write to its own `interested` flag, and read from the other thread's `interested` flag. To gain entry to its critical section, a thread first sets its `interested` flag to `true` so that the other thread can see it wants to enter. Then, it will politely set the turn to the other thread, and busy wait until it sees that either the other thread is not interested, or the other thread has given back the turn. To leave the critical section, a thread simply sets its `interested` flag to false. 

```{code-block} c
:name: listing:sync:cs_peterson
:caption: Critical section entry and exit code for two threads using Peterson's Algorithm; C++-like definition.
struct lock {
    int turn = 0;
    bool interested[2] = {false; false};
    
    void acquire() {
        int me = getTid(); // which thread is running this function? Threads are numbered 0 or 1.
        int other = 1 - me; // other thread's id
        
        interested[me] = true; // this thread wants to enter the critical section
        turn = other;          // but is giving the other thread a chance to go first
        while(interested[other] == true && turn == other) { }; // empty loop body, just spins
        
    }

    void release() {
        int me = getTid();
        interested[me] = false;
    }
}
```

Let's see if this solution meets all of our requirements.

Mutual Exclusion
 : The combination of `interested` and `turn` variables ensure that only one thread can be in the critical section at a time. Suppose both T0 and T1 try to enter their critical sections at nearly the same time by calling `acquire()`. Both set their `interested` flags to `true`, so now both threads can see that the other thread is trying to enter. Now, both threads set `turn` to the other thread's id---either T0 setting `turn = 1` takes effect first and is overwritten by T1 setting `turn = 0`, or vice versa. Only one thread will be able to exit the `while` loop, because `turn` must be either 0 or 1. Suppose T0 is already in the critical section when T1 calls `acquire()`, then `interested[0] == true` and `turn == 1`, and T1 must wait in the `while` loop until `interested[0]` becomes `false`, which only happens when T0 leaves the critical section by calling `release()`.
 
Progress
 : The second part of the progress requirement (threads in their *remainder* code do not delay threads that want to enter the critical section) is satisfied because the `interested` flag is false for any thread in the *remainder* code, and a thread can enter when `interested[other] == false`. The first part of the progress requirement (threads that want to enter can eventually do so) is satisfied because once a thread $T_i$ calls `acquire()` it will eventually return. There are only three possibilities. (1) if the other thread, $T_j$, is not interested in using the critical section, then `interested[other] == false` and $T_i$ can exit the `while` loop immediately and enter the critical section. (2) if the other thread, $T_j$, is already in its critical section, then `interested[other]` will eventually become `false` when $T_j$ leaves its critical section, and $T_i$ can exit its `while` loop and enter. (3) The other thread, $T_j$ is also requesting entry to its critical section, and must set `turn` to $T_i$ after setting its own interested flag; $T_i$ will exit its `while` loop because `turn == other` is now false, and $T_i$ can enter the critical section. 
 
Bounded Waiting
 : Once a thread $T_i$ has indicated interest in entering the critical section, the other thread $T_j$ can enter at most once before $T_i$
 is granted entry, because $T_j$ must set `turn` to $T_i$ when it tries to enter again.
 
We have made an informal argument that Peterson's Algorithm satisfies the requirements. You should be skeptical of such arguments, because the history of synchronization teaches us that informal correctness arguments about concurrency are often wrong. The interested reader can find formal proofs of the correctness of Peterson's algorithm {cite}`Schneider1997`, or you can take our word for it.  

(cont:sync:criticalsection:hw_soln)=
## Critical Section Solution 5 - Hardware Assistance



## Discussion